In [1]:
# !pip install pandas
# !pip install cmake
# !pip install pyarrow
# !pip install streamlit

In [1]:
import IPython
import pandas as pd
import pyarrow.parquet as pq
import gc


In [2]:
# df = pq.read_table('./data/yellow_tripdata_2022-01.parquet').to_pandas()
df = pd.read_csv('./data/uber_data.csv').dropna().astype({
    'VendorID':'int32',
    'tpep_pickup_datetime':'datetime64[ns]', 
    'tpep_dropoff_datetime':'datetime64[ns]',
    'passenger_count':'int16', 
    'trip_distance':'float16', 
    'pickup_longitude':'float16',
    'pickup_latitude':'float16', 
    'RatecodeID':'int16', 
    'store_and_fwd_flag' :'category',
    'dropoff_longitude': 'float16', 
    'dropoff_latitude':'float16', 
    'payment_type':'int16', 
    'fare_amount':'float16',
    'extra':'float16', 
    'mta_tax':'float16', 
    'tip_amount':'float16', 
    'tolls_amount':'float16',
    'improvement_surcharge':'float16', 
    'total_amount':'float16'
}) 

df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2016-03-01,2016-03-01 00:07:55,1,2.500000,-74.0000,40.75000,1,N,-74.0000,40.75000,1,9.0,0.5,0.5,2.050781,0.000000,0.300049,12.351562
1,1,2016-03-01,2016-03-01 00:11:06,1,2.900391,-74.0000,40.78125,1,N,-74.0000,40.71875,1,11.0,0.5,0.5,3.050781,0.000000,0.300049,15.351562
2,2,2016-03-01,2016-03-01 00:31:06,2,19.984375,-73.8125,40.65625,1,N,-74.0000,40.68750,1,54.5,0.5,0.5,8.000000,0.000000,0.300049,63.812500
3,2,2016-03-01,2016-03-01 00:00:00,3,10.781250,-73.8750,40.78125,1,N,-74.0000,40.75000,1,31.5,0.0,0.5,3.779297,5.539062,0.300049,41.625000
4,2,2016-03-01,2016-03-01 00:00:00,5,30.437500,-74.0000,40.78125,3,N,-74.1875,40.68750,1,98.0,0.0,0.0,0.000000,15.500000,0.300049,113.812500


In [4]:
# df.info()

In [3]:
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df = df.drop_duplicates().reset_index(drop=True)
df['trip_id'] = df.index
df.info(memory_usage="deep")
# gc.collect()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 20 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   VendorID               100000 non-null  int32         
 1   tpep_pickup_datetime   100000 non-null  datetime64[ns]
 2   tpep_dropoff_datetime  100000 non-null  datetime64[ns]
 3   passenger_count        100000 non-null  int16         
 4   trip_distance          100000 non-null  float16       
 5   pickup_longitude       100000 non-null  float16       
 6   pickup_latitude        100000 non-null  float16       
 7   RatecodeID             100000 non-null  int16         
 8   store_and_fwd_flag     100000 non-null  category      
 9   dropoff_longitude      100000 non-null  float16       
 10  dropoff_latitude       100000 non-null  float16       
 11  payment_type           100000 non-null  int16         
 12  fare_amount            100000 non-null  float

In [4]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'pickup_longitude',
       'pickup_latitude', 'RatecodeID', 'store_and_fwd_flag',
       'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount',
       'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount', 'trip_id'],
      dtype='object')

In [6]:
datetime_dim = df[['tpep_pickup_datetime','tpep_dropoff_datetime']].reset_index(drop=True)
datetime_dim['tpep_pickup_datetime'] = datetime_dim['tpep_pickup_datetime'] 
datetime_dim['pick_hour'] = datetime_dim['tpep_pickup_datetime'].dt.hour
datetime_dim['pick_day'] = datetime_dim['tpep_pickup_datetime'].dt.day
datetime_dim['pick_month'] = datetime_dim['tpep_pickup_datetime'].dt.month
datetime_dim['pick_year'] = datetime_dim['tpep_pickup_datetime'].dt.year
datetime_dim['pick_weekday'] = datetime_dim['tpep_pickup_datetime'].dt.weekday

datetime_dim['tpep_dropoff_datetime'] = datetime_dim['tpep_dropoff_datetime'] 
datetime_dim['drop_hour'] = datetime_dim['tpep_dropoff_datetime'].dt.hour
datetime_dim['drop_day'] = datetime_dim['tpep_dropoff_datetime'].dt.day
datetime_dim['drop_month'] = datetime_dim['tpep_dropoff_datetime'].dt.month
datetime_dim['drop_year'] = datetime_dim['tpep_dropoff_datetime'].dt.year
datetime_dim['drop_weekday'] = datetime_dim['tpep_dropoff_datetime'].dt.weekday

datetime_dim['datetime_id'] = datetime_dim.index.astype('int32')

# datetime_dim.columns

datetime_dim = datetime_dim[['datetime_id','tpep_pickup_datetime', 'pick_hour', 'pick_day', 'pick_month', 'pick_year', 'pick_weekday',
                             'tpep_dropoff_datetime', 'drop_hour', 'drop_day', 'drop_month', 'drop_year', 'drop_weekday']]

datetime_dim.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 13 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   datetime_id            100000 non-null  int32         
 1   tpep_pickup_datetime   100000 non-null  datetime64[ns]
 2   pick_hour              100000 non-null  int32         
 3   pick_day               100000 non-null  int32         
 4   pick_month             100000 non-null  int32         
 5   pick_year              100000 non-null  int32         
 6   pick_weekday           100000 non-null  int32         
 7   tpep_dropoff_datetime  100000 non-null  datetime64[ns]
 8   drop_hour              100000 non-null  int32         
 9   drop_day               100000 non-null  int32         
 10  drop_month             100000 non-null  int32         
 11  drop_year              100000 non-null  int32         
 12  drop_weekday           100000 non-null  int32

In [7]:
passenger_count_dim = df[['passenger_count']].reset_index(drop=True)
passenger_count_dim['passenger_count_id'] = passenger_count_dim.index.astype('int32')
passenger_count_dim = passenger_count_dim[['passenger_count_id','passenger_count']]
passenger_count_dim.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 2 columns):
 #   Column              Non-Null Count   Dtype
---  ------              --------------   -----
 0   passenger_count_id  100000 non-null  int32
 1   passenger_count     100000 non-null  int16
dtypes: int16(1), int32(1)
memory usage: 586.1 KB


In [8]:
trip_distance_dim = df[['trip_distance']].reset_index(drop=True)
trip_distance_dim['trip_distance_id'] = trip_distance_dim.index.astype('int32')
trip_distance_dim=trip_distance_dim[['trip_distance_id','trip_distance']]
trip_distance_dim.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 2 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   trip_distance_id  100000 non-null  int32  
 1   trip_distance     100000 non-null  float16
dtypes: float16(1), int32(1)
memory usage: 586.1 KB


In [9]:
rate_code_type = {
    1: 'Standard rate',
    2: 'JFK',
    3: 'Newark',
    4: 'Nassau or Westchester',
    5: 'Negotiated fare',
    6: 'Group ride'
}

rate_code_dim = df[['RatecodeID']].reset_index(drop=True)
rate_code_dim['rate_code_id'] = rate_code_dim.index.astype('int32')
rate_code_dim['rate_code_name'] = rate_code_dim['RatecodeID'].map(rate_code_type)
rate_code_dim = rate_code_dim[['rate_code_id','rate_code_name']]
rate_code_dim.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 2 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   rate_code_id    100000 non-null  int32 
 1   rate_code_name  100000 non-null  object
dtypes: int32(1), object(1)
memory usage: 7.0 MB


In [10]:
pickup_location_dim = df[['pickup_latitude','pickup_longitude']].reset_index(drop=True)
pickup_location_dim['pickup_location_id'] = pickup_location_dim.index.astype('int32')
pickup_location_dim=pickup_location_dim[['pickup_location_id','pickup_latitude','pickup_longitude']]
pickup_location_dim.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   pickup_location_id  100000 non-null  int32  
 1   pickup_latitude     100000 non-null  float16
 2   pickup_longitude    100000 non-null  float16
dtypes: float16(2), int32(1)
memory usage: 781.4 KB


In [11]:
dropoff_location_dim = df[['dropoff_latitude','dropoff_longitude']].reset_index(drop=True)
dropoff_location_dim['dropoff_location_id'] = dropoff_location_dim.index.astype('int32')
dropoff_location_dim=dropoff_location_dim[['dropoff_location_id','dropoff_latitude','dropoff_longitude']]
dropoff_location_dim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 3 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   dropoff_location_id  100000 non-null  int32  
 1   dropoff_latitude     100000 non-null  float16
 2   dropoff_longitude    100000 non-null  float16
dtypes: float16(2), int32(1)
memory usage: 781.4 KB


In [12]:
payment_type_name ={
    1: 'Credit card',
    2: 'Cash',
    3: 'No charge',
    4: 'Dispute',
    5: 'Unknown',
    6: 'Voided trip'
}

payment_type_dim = df[['payment_type']].reset_index(drop=True)
payment_type_dim['payment_type_id'] = payment_type_dim.index.astype('int32')
payment_type_dim['payment_type_name'] = payment_type_dim['payment_type'].map(payment_type_name)
payment_type_dim = payment_type_dim[['payment_type_id', 'payment_type', 'payment_type_name']]
payment_type_dim.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 3 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   payment_type_id    100000 non-null  int32 
 1   payment_type       100000 non-null  int16 
 2   payment_type_name  100000 non-null  object
dtypes: int16(1), int32(1), object(1)
memory usage: 6.8 MB


In [16]:
# fact_table = (
#     df.merge(passenger_count_dim, on='passenger_count')
#     .merge(trip_distance_dim, on='trip_distance')
#     .merge(rate_code_dim, on='RatecodeID')
#     .merge(pickup_location_dim, on=['pickup_longitude','pickup_latitude'])
#     .merge(dropoff_location_dim, on=['dropoff_longitude', 'dropoff_latitude'])
#     .merge(datetime_dim, on=['tpep_pickup_datetime','tpep_dropoff_datetime'])
#     .merge(payment_type_dim, on='payment_type')
#     [['trip_id', 'VendorID', 'datetime_id', 'passenger_count_id', 'trip_distance_id', 'rate_code_id'
#       , 'store_and_fwd_flag', 'pickup_location_id', 'dropoff_location_id', 'payment_type_id', 'fare_amount'
#       , 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvment_surcharg', 'total_amount']]
# )

fact_table = df.merge(passenger_count_dim, left_on='trip_id', right_on='passenger_count_id') \
             .merge(trip_distance_dim, left_on='trip_id', right_on='trip_distance_id') \
             .merge(rate_code_dim, left_on='trip_id', right_on='rate_code_id') \
             .merge(pickup_location_dim, left_on='trip_id', right_on='pickup_location_id') \
             .merge(dropoff_location_dim, left_on='trip_id', right_on='dropoff_location_id')\
             .merge(datetime_dim, left_on='trip_id', right_on='datetime_id') \
             .merge(payment_type_dim, left_on='trip_id', right_on='payment_type_id') \
             [['trip_id','VendorID', 'datetime_id', 'passenger_count_id',
               'trip_distance_id', 'rate_code_id', 'store_and_fwd_flag', 'pickup_location_id', 'dropoff_location_id',
               'payment_type_id', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
               'improvement_surcharge', 'total_amount']]
fact_table

,trip_id,VendorID,datetime_id,passenger_count_id,trip_distance_id,rate_code_id,store_and_fwd_flag,pickup_location_id,dropoff_location_id,payment_type_id,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,0,1,0,0,0,0,N,0,0,0,9.0,0.5,0.5,2.050781,0.000000,0.300049,12.351562
1,1,1,1,1,1,1,N,1,1,1,11.0,0.5,0.5,3.050781,0.000000,0.300049,15.351562
2,2,2,2,2,2,2,N,2,2,2,54.5,0.5,0.5,8.000000,0.000000,0.300049,63.812500
3,3,2,3,3,3,3,N,3,3,3,31.5,0.0,0.5,3.779297,5.539062,0.300049,41.625000
4,4,2,4,4,4,4,N,4,4,4,98.0,0.0,0.0,0.000000,15.500000,0.300049,113.812500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99995,1,99995,99995,99995,99995,N,99995,99995,99995,5.0,0.0,0.5,0.000000,0.000000,0.300049,5.800781
99996,99996,1,99996,99996,99996,99996,N,99996,99996,99996,14.0,0.0,0.5,2.000000,0.000000,0.300049,16.796875
99997,99997,1,99997,99997,99997,99997,N,99997,99997,99997,29.0,0.0,0.5,8.796875,5.539062,0.300049,44.125000
99998,99998,2,99998,99998,99998,99998,N,99998,99998,99998,5.5,0.5,0.5,1.360352,0.000000,0.300049,8.156250
